<a href="https://colab.research.google.com/github/ankit-rathi/Data-Science-with-Python/blob/master/Stock_Analysis/Nifty100_ATH_TTM_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install yfinance

In [ ]:
import pandas as pd
import numpy as np

mypf = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Data-Science-with-Python/master/Stock_Analysis/data/Nifty100Scrips.csv')

#mypf = mypf[mypf['Nifty100'] == 1]

stock_n100 = mypf['Symbol'].unique()

exclude = ['ACC','AMBUJACEM','CADILAHC','GILLETTE','NESTLEIND','PGHH','SIEMENS']

stock_n100 = mypf[~mypf['Symbol'].isin(exclude) ]['Symbol']

#stock_ids

stock_ids = stock_n100[71:]
#stock_ids = ['M&M', 'MARUTI']

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

df_analysis = pd.DataFrame()

for stock_name in stock_ids: 


    print(stock_name)

    link_blueprint = "https://www.screener.in/company/"
    
    stock_list = ['AUBANK', 'COLPAL', 'GLAND','WHIRLPOOL']
    
    if stock_name in stock_list:
        actual_link = link_blueprint + stock_name.upper()
    else:
        actual_link = link_blueprint + stock_name.upper()  + "/consolidated/"

    def get_list(actual_link):
        page = requests.get(actual_link)
        link_soup = BeautifulSoup(page.content, 'html.parser')

        test_list = link_soup.find(id="profit-loss").text.replace("  ", "").split('\n')

        while("" in test_list) :
            test_list.remove("")
        while("\r" in test_list) :
            test_list.remove("\r")        
        while(" \r" in test_list) :
            test_list.remove(" \r")

        return test_list

    test_list = get_list(actual_link)

    #print(len(test_list))
    n = len(test_list) 

    if n < 100:
        actual_link = link_blueprint + stock_name.upper() 
        test_list = get_list(actual_link)
        #print(len(test_list))

    q = 0

    if n > 162:
        p = 8
        q = n-162
    elif n > 120 and n <= 162:
        p = 8
    elif n < 120 and n > 100:
        p = 7   

    else:
        p = 6

    df_yoy = pd.DataFrame()
    #print(test_list)
    if 'TTM' in test_list:
        end_year = 'TTM'
    else:
        end_year = 'Mar 2023'
          
    s = test_list.index(end_year) +1 - test_list.index('Mar 2017')

    df_yoy['Year'] = test_list[test_list.index('Mar 2017'):test_list.index(end_year)+1]
    df_yoy['Sales'] = test_list[test_list.index('Expenses\xa0+')-s:test_list.index('Expenses\xa0+')]
    df_yoy['Profit'] = test_list[test_list.index('Tax %')-s:test_list.index('Tax %')]

    df_yoy.set_index('Year', inplace=True)
    df_yoy = df_yoy.T
    df_yoy['Stock'] = stock_name.upper() + '.NS'
    df_yoy
    df_analysis = df_analysis.append(df_yoy)
    
df_analysis

In [ ]:
#df_analysis.columns
cols = [ 'Mar 2017','Mar 2018', 'Mar 2019', 'Mar 2020', 'Mar 2021', 'Mar 2022', 'Mar 2023', 'TTM']
#df_analysis[cols] = df_analysis[cols].apply(pd.to_numeric)
for col in cols:
    df_analysis[col] = df_analysis[col].str.replace(',', '').astype(float)

In [ ]:
print('TTM: ', df_analysis['TTM'].count())
print('Mar 2023: ', df_analysis['Mar 2023'].count())

In [ ]:
df_analysis['Latest']  = df_analysis[["TTM", "Mar 2023"]].max(axis=1)
df_analysis.drop([ "TTM", "Mar 2023"], axis=1, inplace=True)

In [ ]:
df_analysis['Max'] = df_analysis.max(axis=1)
df_analysis['Diff%'] = round((df_analysis['Max'] - df_analysis['Latest'])*100/df_analysis['Max'],2)

In [ ]:
df_tmp = df_analysis[df_analysis['Diff%'] <= 10]
tmp = pd.DataFrame(df_tmp['Stock'].value_counts())
len(tmp[tmp['Stock']>1].index)

In [ ]:
tmp[tmp['Stock']>1].index